In [ ]:
# daily_briefing.ipynb 의 셀 내용

import os
import requests
import feedparser
import pytz
import google.generativeai as genai
from datetime import datetime, timedelta

# ─── 환경변수 (GitHub Actions Secrets에서 가져옴) ─────────────────────
OPENWEATHER_API_KEY = os.getenv("OPENWEATHER_API_KEY")
GEMINI_API_KEY      = os.getenv('GOOGLE_API_KEY')
CITY                = os.getenv("CITY_NAME", "Seoul,KR") # 기본값 설정
DISCORD_WEBHOOK     = os.getenv("DISCORD_WEBHOOK_URL")

# ─── 고정값 설정 ──────────────────────────────────────────────────
NEWS_RSS_URLS = [
    "https://feeds.bbci.co.uk/news/business/rss.xml",
    "https://feeds.bbci.co.uk/news/science_and_environment/rss.xml",
    "https://feeds.bbci.co.uk/news/technology/rss.xml"
]
GAMING_RSS_URLS = [
    "https://webzine.inven.co.kr/news/rss.php",
    "https://www.gamedeveloper.com/rss.xml",
    "https://game.donga.com/feeds/rss/",
    "https://www.gametoc.co.kr/rss/S1N86.xml",
    "https://bbs.ruliweb.com/news/537/rss"
]
TZ = pytz.timezone("Asia/Seoul")
# ──────────────────────────────────────────────────────────────────

# 💡 API 키 설정 및 모델 초기화
# API 키가 없는 경우 에러를 발생시켜 워크플로우를 중단시킴
if not GEMINI_API_KEY:
    raise ValueError("GOOGLE_API_KEY secret is not set.")
if not OPENWEATHER_API_KEY:
    raise ValueError("OPENWEATHER_API_KEY secret is not set.")
if not DISCORD_WEBHOOK:
    raise ValueError("DISCORD_WEBHOOK_URL secret is not set.")

genai.configure(api_key=GEMINI_API_KEY)
model = genai.GenerativeModel('gemini-1.5-flash')

# (이전 답변에서 제공한 모든 함수들을 여기에 붙여넣으세요)
# fetch_weather(), create_temperature_graph(), build_weather_embed()
# fetch_recent_entries(), summarize_news_with_gemini(), build_news_embed()
# build_gaming_news_embed(), build_gaming_trends_embed(), send_to_discord()
# ...
# ... (아래는 예시로 몇 개만 다시 첨부합니다) ...

def fetch_weather():
    url = "https://api.openweathermap.org/data/2.5/weather"
    params = {"q": CITY, "appid": OPENWEATHER_API_KEY, "units": "metric", "lang": "kr"}
    r = requests.get(url, params=params); r.raise_for_status()
    current = r.json()

    forecast_url = "https://api.openweathermap.org/data/2.5/forecast"
    r = requests.get(forecast_url, params=params); r.raise_for_status()
    forecast = r.json()

    hourly_temps = []
    now = datetime.now(TZ)
    for item in forecast['list']:
        dt = datetime.fromtimestamp(item['dt'], TZ)
        if dt <= now + timedelta(hours=24):
            hourly_temps.append({
                "time": dt.strftime("%H:%M"),
                "temp": item['main']['temp'],
                "icon": item['weather'][0]['icon']
            })

    return {
        "current": {
            "desc": current["weather"][0]["description"].capitalize(),
            "temp": current["main"]["temp"],
            "feels": current["main"]["feels_like"],
            "humidity": current["main"]["humidity"],
            "icon": current["weather"][0]["icon"],
        },
        "hourly": hourly_temps
    }

def send_to_discord(embeds):
    for embed in embeds:
        payload = {"embeds": [embed]}
        try:
            r = requests.post(DISCORD_WEBHOOK, json=payload)
            r.raise_for_status()
        except Exception as e:
            print(f"Error sending embed to Discord: {e}")
            continue

# ... (나머지 모든 함수 정의) ...

# 💡 메인 실행 함수 (기존과 동일)
def run_daily_briefing():
    print("✅ Daily briefing script started...")

    # 날씨
    print("🌦️ Fetching weather...")
    try:
        wdata = fetch_weather()
        wembed = build_weather_embed(wdata)
        send_to_discord([wembed])
        print("👍 Weather info sent!")
    except Exception as e:
        print(f"❌ Error in weather processing: {e}")
    # ... (뉴스, 게임 뉴스 로직도 동일하게 포함) ...
    print("🎉 All tasks completed!")

# 💥 스크립트 실행
if __name__ == "__main__":
    run_daily_briefing()